In [1]:
import math
import warnings
from typing import Dict, Literal
import pandas as pd
warnings.simplefilter("ignore")
import delu  
import numpy as np
import scipy.special
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import sklearn.preprocessing
import torch
import torch.nn.functional as F
import torch.optim
from torch import Tensor
from tqdm.std import tqdm
from pandas import read_csv
warnings.resetwarnings()

from rtdl_revisiting_models import MLP, ResNet, FTTransformer

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Set random seeds in all libraries.
delu.random.seed(0)

0

In [3]:
dataset = pd.read_excel(r'D:\Data\NHRI\NHDF\Train_set.xlsx')
print(type(dataset))
dataset.head()
values = dataset.values
values = values.astype('float32')
print(type(values))
#print(values[0:5])
values = np.array(values)
print(values[0:5])
print(values.shape)


X_train = values[:, 0:10]
y_train = values[:, -1]
print(X_train.shape)
print(y_train.shape)

print(type(X_train))
print(type(y_train))

print("X_train.shape: ", X_train.shape)
print("y_train.shape: ", y_train.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
[[5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 1.7200001e+01
  7.5776299e+01]
 [5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 2.0000000e+01
  9.4489334e+01]
 [5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 2.2500000e+01
  1.0625959e+02]
 [5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 2.5000000e+01
  1.2227732e+02]
 [5.4000001e+00 5.1999998e+00 9.4999999e-01 4.0000000e+00 1.0000000e+00
  0.0000000e+00 0.0000000e+00 1.0000000e-01 3.0070579e-01 2.8200001e+01
  1.3469843e+02]]
(600, 11)
(600, 10)
(600,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
X_train.shape:  (600, 10)
y_train.shape:  (600,)


In [4]:
# >>> Dataset.
TaskType = Literal["regression", "binclass", "multiclass"]

task_type: TaskType = "regression"
n_classes = None
dataset = sklearn.datasets.fetch_california_housing()
X_cont: np.ndarray = X_train  #dataset["data"]
Y: np.ndarray = y_train       #dataset["target"]

In [5]:
label = '底缘单宽净动水荷载'

# NOTE: input test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\Test_set.csv', encoding = 'utf-8') #, header = None
y_test = test_data[label]
x_test = test_data.drop(columns=[label])
print(type(y_test))
print(type(x_test))
y_test = np.array(y_test)
x_test = np.array(x_test)
print(type(y_test))
print(type(x_test))
print(y_test.shape)
print(x_test.shape)

<class 'pandas.core.series.Series'>
<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(72,)
(72, 10)


In [6]:
# NOTE: read 1st test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\1st_test.csv', encoding = 'utf-8') #, header = None
y_test1 = test_data[label]
x_test1 = test_data.drop(columns=[label])
y_test1 = np.array(y_test1)
x_test1 = np.array(x_test1)
print(y_test1.shape)
print(x_test1.shape)

# NOTE: read 2nd test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\2st_test.csv', encoding = 'utf-8') #, header = None
y_test2 = test_data[label]
x_test2 = test_data.drop(columns=[label])
y_test2 = np.array(y_test2)
x_test2 = np.array(x_test2)
print(y_test2.shape)
print(x_test2.shape)

# NOTE: read 3rd test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\3st_test.csv', encoding = 'utf-8') #, header = None
y_test3 = test_data[label]
x_test3 = test_data.drop(columns=[label])
y_test3 = np.array(y_test3)
x_test3 = np.array(x_test3)
print(y_test3.shape)
print(x_test3.shape)

# NOTE: read 4th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\4st_test.csv', encoding = 'utf-8') #, header = None
y_test4 = test_data[label]
x_test4 = test_data.drop(columns=[label])
y_test4 = np.array(y_test4)
x_test4 = np.array(x_test4)
print(y_test4.shape)
print(x_test4.shape)

# NOTE: read 5th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\5st_test.csv', encoding = 'utf-8') #, header = None
y_test5 = test_data[label]
x_test5 = test_data.drop(columns=[label])
y_test5 = np.array(y_test5)
x_test5 = np.array(x_test5)
print(y_test5.shape)
print(x_test5.shape)

# NOTE: read 6th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\6st_test.csv', encoding = 'utf-8') #, header = None
y_test6 = test_data[label]
x_test6 = test_data.drop(columns=[label])
y_test6 = np.array(y_test6)
x_test6 = np.array(x_test6)
print(y_test6.shape)
print(x_test6.shape)

# NOTE: read 7th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\7st_test.csv', encoding = 'utf-8') #, header = None
y_test7 = test_data[label]
x_test7 = test_data.drop(columns=[label])
y_test7 = np.array(y_test7)
x_test7 = np.array(x_test7)
print(y_test7.shape)
print(x_test7.shape)

# NOTE: read 8th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\8st_test.csv', encoding = 'utf-8') #, header = None
y_test8 = test_data[label]
x_test8 = test_data.drop(columns=[label])
y_test8 = np.array(y_test8)
x_test8 = np.array(x_test8)
print(y_test8.shape)
print(x_test8.shape)

# NOTE: read 9th test set
test_data = read_csv(r'D:\Data\NHRI\NHDF\9st_test.csv', encoding = 'utf-8') #, header = None
y_test9 = test_data[label]
x_test9 = test_data.drop(columns=[label])
y_test9 = np.array(y_test9)
x_test9 = np.array(x_test9)
print(y_test9.shape)
print(x_test9.shape)

(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)
(8,)
(8, 10)


## Dataset


In [7]:
# >>> Continuous features.
X_cont: np.ndarray = X_cont.astype(np.float32)
n_cont_features = X_cont.shape[1]

# >>> Categorical features.
# NOTE: the above datasets do not have categorical features, but,
# for the demonstration purposes, it is possible to generate them.
cat_cardinalities = [
    # NOTE: uncomment the two lines below to add two categorical features.
    # 4,  # Allowed values: [0, 1, 2, 3].
    # 7,  # Allowed values: [0, 1, 2, 3, 4, 5, 6].
]
X_cat = (
    np.column_stack(
        [np.random.randint(0, c, (len(X_cont),)) for c in cat_cardinalities]
    )
    if cat_cardinalities
    else None
)

# >>> Labels.
# Regression labels must be represented by float32.
if task_type == "regression":
    Y = Y.astype(np.float32)
else:
    assert n_classes is not None
    Y = Y.astype(np.int64)
    assert set(Y.tolist()) == set(
        range(n_classes)
    ), "Classification labels must form the range [0, 1, ..., n_classes - 1]"

# >>> Split the dataset.
all_idx = np.arange(len(Y))

train_idx, val_idx = sklearn.model_selection.train_test_split(  
    all_idx, train_size=0.8
)



data_numpy = {
    "train": {"x_cont": X_cont[train_idx], "y": Y[train_idx]},
    "val": {"x_cont": X_cont[val_idx], "y": Y[val_idx]},
    "test": {"x_cont": x_test.astype(np.float32), "y": y_test.astype(np.float32)},
    'test1': {'x_cont': x_test1.astype(np.float32), 'y': y_test1.astype(np.float32)},
    'test2': {'x_cont': x_test2.astype(np.float32), 'y': y_test2.astype(np.float32)},
    'test3': {'x_cont': x_test3.astype(np.float32), 'y': y_test3.astype(np.float32)},
    'test4': {'x_cont': x_test4.astype(np.float32), 'y': y_test4.astype(np.float32)},
    'test5': {'x_cont': x_test5.astype(np.float32), 'y': y_test5.astype(np.float32)},
    'test6': {'x_cont': x_test6.astype(np.float32), 'y': y_test6.astype(np.float32)},
    'test7': {'x_cont': x_test7.astype(np.float32), 'y': y_test7.astype(np.float32)},
    'test8': {'x_cont': x_test8.astype(np.float32), 'y': y_test8.astype(np.float32)},
    'test9': {'x_cont': x_test9.astype(np.float32), 'y': y_test9.astype(np.float32)},
    'train_NoShuffle' : {'x_cont': X_train.astype(np.float32), 'y': y_train.astype(np.float32)}
}
if X_cat is not None:
    data_numpy["train"]["x_cat"] = X_cat[train_idx]
    data_numpy["val"]["x_cat"] = X_cat[val_idx]
    data_numpy["test"]["x_cat"] = X_cat[test_idx]

## Preprocessing


In [8]:
# >>> Feature preprocessing.
# NOTE
# Fancy preprocessing strategy.
# The noise is added to improve the output of QuantileTransformer in some cases.
X_cont_train_numpy = data_numpy["train"]["x_cont"]
noise = (
    np.random.default_rng(0)
    .normal(0.0, 1e-5, X_cont_train_numpy.shape)
    .astype(X_cont_train_numpy.dtype)
)
preprocessing = sklearn.preprocessing.QuantileTransformer(
    n_quantiles=max(min(len(train_idx) // 30, 1000), 10),
    output_distribution="normal",
    subsample=10**9,
).fit(X_cont_train_numpy + noise)
del X_cont_train_numpy

for part in data_numpy:
    data_numpy[part]["x_cont"] = preprocessing.transform(data_numpy[part]["x_cont"])

# >>> Label preprocessing.
if task_type == "regression":
    Y_mean = data_numpy["train"]["y"].mean().item()
    Y_std = data_numpy["train"]["y"].std().item()
    for part in data_numpy:
        data_numpy[part]["y"] = (data_numpy[part]["y"] - Y_mean) / Y_std

# >>> Convert data to tensors.
data = {
    part: {k: torch.as_tensor(v, device=device) for k, v in data_numpy[part].items()}
    for part in data_numpy
}

if task_type != "multiclass":
    # Required by F.binary_cross_entropy_with_logits
    for part in data:
        data[part]["y"] = data[part]["y"].float()

## Model


In [9]:
# The output size.
d_out = n_classes if task_type == "multiclass" else 1

model = FTTransformer(
    n_cont_features=n_cont_features,
    cat_cardinalities=cat_cardinalities,
    d_out=d_out,
    **FTTransformer.get_default_kwargs(),
).to(device)
optimizer = model.make_default_optimizer()

## Training


In [11]:
def apply_model(batch: Dict[str, Tensor]) -> Tensor:
    if isinstance(model, (MLP, ResNet)):
        x_cat_ohe = (
            [
                F.one_hot(column, cardinality)
                for column, cardinality in zip(batch["x_cat"].T, cat_cardinalities)
            ]
            if "x_cat" in batch
            else []
        )
        return model(torch.column_stack([batch["x_cont"]] + x_cat_ohe)).squeeze(-1)

    elif isinstance(model, FTTransformer):
        return model(batch["x_cont"], batch.get("x_cat")).squeeze(-1)

    else:
        raise RuntimeError(f"Unknown model type: {type(model)}")


loss_fn = (
    F.binary_cross_entropy_with_logits
    if task_type == "binclass"
    else F.cross_entropy
    if task_type == "multiclass"
    else F.mse_loss
)


@torch.no_grad()
def evaluate(part: str) -> float:
    model.eval()

    eval_batch_size = 2 #8096
    y_pred = (
        torch.cat(
            [
                apply_model(batch)
                for batch in delu.iter_batches(data[part], eval_batch_size)
            ]
        )
        .cpu()
        .numpy()
    )
    y_true = data[part]["y"].cpu().numpy()

    if task_type == "binclass":
        y_pred = np.round(scipy.special.expit(y_pred))
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
    elif task_type == "multiclass":
        y_pred = y_pred.argmax(1)
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
    else:
        assert task_type == "regression"
        score = -(sklearn.metrics.mean_squared_error(y_true, y_pred) ** 0.5 * Y_std)
    return score  # The higher -- the better.


#print(f'Test score before training: {evaluate("test"):.4f}')
print(f'Val MSE score before training: {evaluate("val"):.4f}')        
print(f'Test MSE score before training: {evaluate("test"):.4f}')      
print(f'Test 1  MSE score before training: {evaluate("test1"):.4f}')  
print(f'Test 2  MSE score before training: {evaluate("test2"):.4f}')  
print(f'Test 3  MSE score before training: {evaluate("test3"):.4f}')  
print(f'Test 4  MSE score before training: {evaluate("test4"):.4f}')  
print(f'Test 5  MSE score before training: {evaluate("test5"):.4f}')  
print(f'Test 6  MSE score before training: {evaluate("test6"):.4f}')  
print(f'Test 7  MSE score before training: {evaluate("test7"):.4f}')  
print(f'Test 8  MSE score before training: {evaluate("test8"):.4f}')  
print(f'Test 9  MSE score before training: {evaluate("test9"):.4f}')  

Val MSE score before training: -75.7465
Test MSE score before training: -75.8240
Test 1  MSE score before training: -61.7293
Test 2  MSE score before training: -88.4645
Test 3  MSE score before training: -132.3993
Test 4  MSE score before training: -31.4370
Test 5  MSE score before training: -23.2837
Test 6  MSE score before training: -37.1504
Test 7  MSE score before training: -76.0459
Test 8  MSE score before training: -110.6286
Test 9  MSE score before training: -40.5615


## Training


In [12]:
import time, datetime
from datetime import datetime
import os
from datetime import timedelta 

In [13]:
best = {
    'val': -math.inf,
    'test': -math.inf,
    'epoch': -1,
}

trainloss_list = []
valloss_list = []
testloss_list = []
testloss1_list = []
testloss2_list = []
testloss3_list = []
testloss4_list = []
testloss5_list = []
testloss6_list = []
testloss7_list = []
testloss8_list = []
testloss9_list = []

In [15]:
# For demonstration purposes (fast training and bad performance),
# one can set smaller values:
# n_epochs = 20
# patience = 2
n_epochs = 300 #1_000_000_000
patience = 300#16

batch_size = 2 #256
epoch_size = math.ceil(len(train_idx) / batch_size)
timer = delu.tools.Timer()
early_stopping = delu.tools.EarlyStopping(patience, mode="max")
#best = {
#    "val": -math.inf,
#    "test": -math.inf,
#    "epoch": -1,
#}

print(f"Device: {device.type.upper()}")
print("-" * 88 + "\n")
timer.run()
for epoch in range(n_epochs):
    for batch in tqdm(
        delu.iter_batches(data["train"], batch_size, shuffle=True),
        desc=f"Epoch {epoch}",
        total=epoch_size,
    ):
        model.train()
        optimizer.zero_grad()
        loss = loss_fn(apply_model(batch), batch["y"])
        loss.backward()
        optimizer.step()

    #val_score = evaluate("val")
    #test_score = evaluate("test")
    #print(f"(val) {val_score:.4f} (test) {test_score:.4f} [time] {timer}")

    val_score = evaluate('val'); valloss_list.append(val_score)  
    test_score = evaluate('test'); testloss_list.append(test_score)
    test1_score = evaluate("test1"); testloss1_list.append(test1_score)
    test2_score = evaluate("test2"); testloss2_list.append(test2_score)
    test3_score = evaluate("test3"); testloss3_list.append(test3_score)
    test4_score = evaluate("test4"); testloss4_list.append(test4_score)
    test5_score = evaluate("test5"); testloss5_list.append(test5_score)
    test6_score = evaluate("test6"); testloss6_list.append(test6_score)
    test7_score = evaluate("test7"); testloss7_list.append(test7_score)
    test8_score = evaluate("test8"); testloss8_list.append(test8_score)
    test9_score = evaluate("test9"); testloss9_list.append(test9_score)
    print(f'(val) {val_score:.4f} (test) {test_score:.4f} (test1) {test1_score:.4f} (test2) {test2_score:.4f} (test3) {test3_score:.4f} (test4) {test4_score:.4f} (test5) {test5_score:.4f} (test6) {test6_score:.4f} (test7) {test7_score:.4f} (test8) {test8_score:.4f} (test9) {test9_score:.4f}')

    early_stopping.update(val_score)
    if early_stopping.should_stop():
        break

    #if val_score > best["val"]:
    #    print("🌸 New best epoch! 🌸")
    #    best = {"val": val_score, "test": test_score, "epoch": epoch}

    if ((val_score > best['val']) and (test_score > best['test'])):
        print('🌸 New best epoch! 🌸')
        best = {'val': val_score, 'test': test_score, 'epoch': epoch}
        remaining_patience = patience
        save_dir = r'D:\Data\NHRI\NHDF\best model'
        last_save_path = os.path.join(save_dir, f'epoch.{epoch:04d}-val_loss.{val_score:.4f}-test_loss.{test_score:.4f}.pth')    
        torch.save(model, last_save_path) 


    print()

print("\n\nResult:")
print(best)


# save train process  
t = time.localtime()
t.tm_hour, t.tm_min, t.tm_sec    
outtime = str(datetime.today().date()) + ' ' + str(t.tm_hour) + 'H' + str(t.tm_min) + 'M' 
csv_path = r'D:\Data\NHRI\NHDF\train_process\TabM_train_process ' + outtime + '.csv'
loss_hist = np.column_stack((valloss_list, testloss_list, testloss1_list, testloss2_list, testloss3_list, testloss4_list, testloss5_list, testloss6_list, testloss7_list))
np.savetxt(csv_path, loss_hist, delimiter=",", header="val_loss,test_loss,1st_loss,2nd_loss,3rd_loss,4th_loss,5th_loss,6th_loss,7th_loss,8th_loss,9th_loss", encoding='utf-8-sig', comments='')


Device: CUDA
----------------------------------------------------------------------------------------



Epoch 0: 100%|██████████| 240/240 [00:01<00:00, 161.25it/s]


(val) -5.5008 (test) -6.1548 (test1) -5.7573 (test2) -4.4615 (test3) -6.4793 (test4) -2.9573 (test5) -3.8020 (test6) -6.8468 (test7) -9.0876 (test8) -5.4248 (test9) -7.9878



Epoch 1: 100%|██████████| 240/240 [00:01<00:00, 185.23it/s]


(val) -5.3741 (test) -9.5195 (test1) -17.2588 (test2) -2.9836 (test3) -4.3268 (test4) -3.4960 (test5) -4.4987 (test6) -7.8490 (test7) -6.0827 (test8) -7.1531 (test9) -17.5459



...

(val) -4.8469 (test) -7.0804 (test1) -8.0064 (test2) -2.8746 (test3) -4.3335 (test4) -3.0422 (test5) -2.8441 (test6) -6.2369 (test7) -6.8968 (test8) -6.3739 (test9) -14.6837



Epoch 259: 100%|██████████| 240/240 [00:01<00:00, 179.92it/s]


(val) -4.7536 (test) -8.1853 (test1) -7.2756 (test2) -2.7118 (test3) -4.2484 (test4) -3.9979 (test5) -5.1992 (test6) -7.8732 (test7) -10.4043 (test8) -5.9092 (test9) -16.6278



Epoch 260:  90%|████████▉ | 215/240 [00:01<00:00, 181.54it/s]


KeyboardInterrupt: 

## 读取保存的最优模型，进一步分析在测试集的预测结果


In [16]:
loaded_model = torch.load(r'D:\Data\NHRI\NHDF\best model\epoch.0132-val_loss.-4.6015-test_loss.-6.0238.pth')
print(loaded_model)
from torchsummary import summary
#summary(loaded_model, input_size= (1, 10))

FTTransformer(
  (cls_embedding): _CLSEmbedding()
  (cont_embeddings): LinearEmbeddings()
  (backbone): FTTransformerBackbone(
    (blocks): ModuleList(
      (0): ModuleDict(
        (attention): MultiheadAttention(
          (W_q): Linear(in_features=192, out_features=192, bias=True)
          (W_k): Linear(in_features=192, out_features=192, bias=True)
          (W_v): Linear(in_features=192, out_features=192, bias=True)
          (W_out): Linear(in_features=192, out_features=192, bias=True)
          (dropout): Dropout(p=0.2, inplace=False)
        )
        (attention_residual_dropout): Dropout(p=0.0, inplace=False)
        (ffn_normalization): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
        (ffn): Sequential(
          (linear1): Linear(in_features=192, out_features=512, bias=True)
          (activation): _ReGLU()
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=256, out_features=192, bias=True)
        )
        (ffn_residua

C:\Users\Administrator\AppData\Local\Temp\ipykernel_4044\3751165959.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(r'D:\Data\NHRI\小论文4\2、模型\2、

In [17]:
# modify to best model
def lz_apply_model(batch: Dict[str, Tensor]) -> Tensor:
    if isinstance(loaded_model, (MLP, ResNet)):
        x_cat_ohe = (
            [
                F.one_hot(column, cardinality)
                for column, cardinality in zip(batch["x_cat"].T, cat_cardinalities)
            ]
            if "x_cat" in batch
            else []
        )
        return loaded_model(torch.column_stack([batch["x_cont"]] + x_cat_ohe)).squeeze(-1)

    elif isinstance(loaded_model, FTTransformer):
        return loaded_model(batch["x_cont"], batch.get("x_cat")).squeeze(-1)

    else:
        raise RuntimeError(f"Unknown loaded_model type: {type(loaded_model)}")

In [18]:
def safe_mape(y_true, y_pred, epsilon=1e-8, threshold=0.01):

    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    
    y_pred = y_pred.to(y_true.device)
    
    
    mask = torch.abs(y_true) > threshold
    
    if mask.sum() == 0:
        return torch.tensor(float('nan'), device=y_true.device)
    
    ape = torch.abs((y_true[mask] - y_pred[mask]) / (y_true[mask] + epsilon))
    
    return torch.mean(ape) * 100

In [19]:

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
@torch.no_grad()
def lz_evaluate(part: str) -> float:
    model.eval()

    eval_batch_size = 2 #8096
    y_pred = (
        torch.cat(
            [
                lz_apply_model(batch)
                for batch in delu.iter_batches(data[part], eval_batch_size)
            ]
        )
        .cpu()
        .numpy()
    )
    
    

    y_true = data[part]["y"].cpu().numpy()
    
    y_true_inv = y_true * Y_std + Y_mean   


    if task_type == "binclass":
        y_pred = np.round(scipy.special.expit(y_pred))
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
    elif task_type == "multiclass":
        y_pred = y_pred.argmax(1)
        score = sklearn.metrics.accuracy_score(y_true, y_pred)
    else:
        assert task_type == "regression"
        score = -(sklearn.metrics.mean_squared_error(y_true, y_pred) ** 0.5 * Y_std)
        # *****************************************************************
        y_pred_inv = y_pred * Y_std + Y_mean
        
        for pred_inv in y_pred_inv:
            print(pred_inv)


        loss_RMSE = ((sklearn.metrics.mean_squared_error(y_true_inv, y_pred_inv) ** 0.5))
        loss_MSE = (sklearn.metrics.mean_squared_error(y_true_inv, y_pred_inv))
        loss_r2 = r2_score(y_true_inv, y_pred_inv)
        loss_MAPE = safe_mape(y_true_inv, y_pred_inv).item()
        loss_MAE = mean_absolute_error(y_true_inv, y_pred_inv)
        print("")
        print("Mean Absolute Error (MAE):", loss_MAE)
        print("Mean Squared Error (MSE):", loss_MSE)
        print("Root Mean Squared Error (RMSE):", loss_RMSE)
        print("Mean Absolute Percentage Error (MAPE):", loss_MAPE, "%")
        print("R² Score:", loss_r2)


    # Y_mean = data_numpy["train"]["y"].mean().item()
    # Y_std = data_numpy["train"]["y"].std().item()
    

In [20]:
# NOTE train set
test_score = lz_evaluate('train_NoShuffle')

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -2.826202082768679
反标准化之后的预测值：
76.59282
92.78865
108.55454
122.2315
134.52296
151.10405
133.67798
153.56941
183.00044
199.91805
228.65697
252.22067
164.49751
189.63979
222.54712
250.43118
280.11722
310.1905
174.75664
197.90782
236.6654
263.5904
290.0487
324.5654
162.45474
189.87918
222.19568
254.40402
281.5177
311.8412
158.14558
177.6825
208.65637
232.03781
262.0278
281.99762
125.687515
146.30116
169.70856
187.92937
211.68666
233.3385
88.495255
97.974625
115.36451
127.691376
146.25249
164.52425
30.536156
50.707817
57.6567
68.36287
76.40083
82.64048
68.279854
90.08954
101.39949
116.39831
131.9133
147.30424
84.15977
125.9938
142.99976
169.17485
189.36653
201.73859
95.35922
140.73988
172.79927
191.76291
214.47229
226.48062
92.09497
134.4755
160.34077
186.19531
204.17339
222.08289
84.27076
123.276596
136.09372
164.41125
177.55463
207.5951
67.45925
91.59967
105.67973
122.75006
137.0391
157.32129
31.427383
53.946632
61.150032
69.41211
73.90201

In [21]:
# NOTE all test set
test_score = lz_evaluate('test')

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -6.023788756255487
反标准化之后的预测值：
49.595894
104.27472
132.44348
139.12183
133.55832
126.51304
101.37189
49.837578
40.346962
80.08217
106.28309
117.036545
113.42769
103.536835
83.06593
46.130493
31.487617
48.91301
69.46789
76.99627
67.66161
58.555977
44.034126
25.01336
120.49992
136.0482
147.158
138.01907
166.68463
142.6429
118.67339
110.29454
144.18634
160.31596
166.55063
155.338
189.82014
169.40096
131.63792
125.78457
176.01396
194.87282
211.64258
187.57101
217.37419
198.17561
170.31999
151.92654
225.03302
243.5061
256.7184
257.4711
255.22128
233.35327
196.92691
150.9816
183.95796
272.77316
293.85373
297.2611
283.10913
263.4435
205.05476
161.55923
144.59065
192.85855
193.48471
189.22469
210.24936
200.4569
158.79309
129.66057

Mean Absolute Error (MAE): 4.44242000579834
Mean Squared Error (MSE): 36.28603744506836
Root Mean Squared Error (RMSE): 6.023789292884368
Mean Absolute Percentage Error (MAPE): 3.644824743270874 %
R² Score: 0.99183475

In [22]:
# NOTE 1st test set
test1_score = lz_evaluate("test1");

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -8.774117954873239
反标准化之后的预测值：
49.595894
104.27472
132.44348
139.12183
133.55832
126.51304
101.37189
49.837578

Mean Absolute Error (MAE): 5.484469413757324
Mean Squared Error (MSE): 76.9851303100586
Root Mean Squared Error (RMSE): 8.774117067264296
Mean Absolute Percentage Error (MAPE): 7.413683891296387 %
R² Score: 0.8970237374305725


In [23]:
# NOTE 2nd test set
test2_score = lz_evaluate("test2");

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -5.151245806028871
反标准化之后的预测值：
40.346962
80.08217
106.28309
117.036545
113.42769
103.536835
83.06593
46.130493

Mean Absolute Error (MAE): 4.539660453796387
Mean Squared Error (MSE): 26.53533172607422
Root Mean Squared Error (RMSE): 5.1512456480034245
Mean Absolute Percentage Error (MAPE): 5.055177211761475 %
R² Score: 0.9691453576087952


In [24]:
# NOTE 3rd test set
test3_score = lz_evaluate("test3");

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -2.7351387821796056
反标准化之后的预测值：
31.487617
48.91301
69.46789
76.99627
67.66161
58.555977
44.034126
25.01336

Mean Absolute Error (MAE): 2.3694543838500977
Mean Squared Error (MSE): 7.480978965759277
Root Mean Squared Error (RMSE): 2.735137833045947
Mean Absolute Percentage Error (MAPE): 5.174879550933838 %
R² Score: 0.9748899936676025


In [25]:
# NOTE  4th test set
test4_score = lz_evaluate("test4")

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -3.0074072766900275
反标准化之后的预测值：
120.49992
136.0482
147.158
138.01907
166.68463
142.6429
118.67339
110.29454

Mean Absolute Error (MAE): 2.61138916015625
Mean Squared Error (MSE): 9.044509887695312
Root Mean Squared Error (RMSE): 3.0074091653274104
Mean Absolute Percentage Error (MAPE): 2.004476547241211 %
R² Score: 0.9719414114952087


In [26]:
# NOTE 5th test set
test5_score = lz_evaluate("test5")

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -3.270800324079498
反标准化之后的预测值：
144.18634
160.31596
166.55063
155.338
189.82014
169.40096
131.63792
125.78457

Mean Absolute Error (MAE): 2.637186050415039
Mean Squared Error (MSE): 10.698150634765625
Root Mean Squared Error (RMSE): 3.2708027508190747
Mean Absolute Percentage Error (MAPE): 1.6942756175994873 %
R² Score: 0.9688020944595337


In [27]:
# NOTE 6th test set
test6_score = lz_evaluate("test6")

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -6.539335513922283
反标准化之后的预测值：
176.01396
194.87282
211.64258
187.57101
217.37419
198.17561
170.31999
151.92654

Mean Absolute Error (MAE): 5.066568374633789
Mean Squared Error (MSE): 42.762939453125
Root Mean Squared Error (RMSE): 6.539337845158713
Mean Absolute Percentage Error (MAPE): 2.595335006713867 %
R² Score: 0.887263298034668


In [28]:
# NOTE  7th test set
test7_score = lz_evaluate("test7")

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -5.272776884950387
反标准化之后的预测值：
225.03302
243.5061
256.7184
257.4711
255.22128
233.35327
196.92691
150.9816

Mean Absolute Error (MAE): 4.331808090209961
Mean Squared Error (MSE): 27.80217170715332
Root Mean Squared Error (RMSE): 5.272776470433136
Mean Absolute Percentage Error (MAPE): 1.9791204929351807 %
R² Score: 0.9724277257919312


In [29]:
# NOTE 8th test set
test8_score = lz_evaluate("test8")

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -5.9570901207537155
反标准化之后的预测值：
183.95796
272.77316
293.85373
297.2611
283.10913
263.4435
205.05476
161.55923

Mean Absolute Error (MAE): 5.293363571166992
Mean Squared Error (MSE): 35.48697280883789
Root Mean Squared Error (RMSE): 5.957094325998027
Mean Absolute Percentage Error (MAPE): 2.3639731407165527 %
R² Score: 0.987386167049408


In [30]:
# NOTE  9th test set
test9_score = lz_evaluate("test9")

输出预测的返回值：
输出相应的真实值：
反标准化之后的真实值：
自带的损失函数 RMSE? ：  -9.475134798430426
反标准化之后的预测值：
144.59065
192.85855
193.48471
189.22469
210.24936
200.4569
158.79309
129.66057

Mean Absolute Error (MAE): 7.647878646850586
Mean Squared Error (MSE): 89.77813720703125
Root Mean Squared Error (RMSE): 9.475132569364465
Mean Absolute Percentage Error (MAPE): 4.522495269775391 %
R² Score: 0.8459529876708984
